# Task for Today  

***

## Spotify Song Popularity Prediction  

Given *data about Spotify songs from 1921-2020*, let's try to predict the **popularity** of a given song.  
  
We will use a linear regression models to make our predictions, but we will focus on outlier detection and removal.

# Getting Started

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

In [ ]:
data = pd.read_csv('../input/spotify-dataset-19212020-160k-tracks/data.csv')

In [ ]:
data

In [ ]:
data = data.drop(['artists', 'id', 'name', 'release_date'], axis=1)

In [ ]:
data.info()

In [ ]:
data

# Outlier Detection

In [ ]:
plt.figure(figsize=(16, 10))

for i in range(len(data.columns)):
    plt.subplot(3, 5, i + 1)
    sns.boxplot(data[data.columns[i]])

plt.show()

In [ ]:
def get_outlier_counts(df, threshold):
    df = df.copy()
    
    # Get the z-score for specified threshold
    threshold_z_score = stats.norm.ppf(threshold)
    
    # Get the z-scores for each value in df
    z_score_df = pd.DataFrame(np.abs(stats.zscore(df)), columns=df.columns)
    
    # Compare df z_scores to the threshold and return the count of outliers in each column
    return (z_score_df > threshold_z_score).sum(axis=0)

In [ ]:
get_outlier_counts(data, 0.99999999999)

In [ ]:
def remove_outliers(df, threshold):
    df = df.copy()
    
    # Get the z-score for specified threshold
    threshold_z_score = stats.norm.ppf(threshold)
    
    # Get the z-scores for each value in df
    z_score_df = pd.DataFrame(np.abs(stats.zscore(df)), columns=df.columns)
    z_score_df = z_score_df > threshold_z_score
    
    # Get indices of the outliers
    outliers = z_score_df.sum(axis=1)
    outliers = outliers > 0
    outlier_indices = df.index[outliers]
    
    # Drop outlier examples
    df = df.drop(outlier_indices, axis=0).reset_index(drop=True)
    
    return df

# Preprocessing

In [ ]:
def preprocess_inputs(df, outliers=True, threshold=0.95):
    df = df.copy()
    
    # Remove outliers if specified
    if outliers == False:
        df = remove_outliers(df, threshold)
    
    # Split df into X and y
    y = df['popularity'].copy()
    X = df.drop('popularity', axis=1).copy()
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

In [ ]:
outlier_X_train, outlier_X_test, outlier_y_train, outlier_y_test = preprocess_inputs(data, outliers=True)

X_train, X_test, y_train, y_test = preprocess_inputs(data, outliers=False, threshold=0.99999999999)

# Training

In [ ]:
# With outliers

outlier_model = LinearRegression()
outlier_model.fit(outlier_X_train, outlier_y_train)

outlier_model_acc = outlier_model.score(outlier_X_test, outlier_y_test)

print("Test Accuracy (Outliers): {:.5f}%".format(outlier_model_acc * 100))

In [ ]:
# Without outliers

model = LinearRegression()
model.fit(X_train, y_train)

model_acc = model.score(X_test, y_test)

print("Test Accuracy (No Outliers): {:.5f}%".format(model_acc * 100))

# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/LQlTO5uhCQ8